In [11]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]


In [2]:
# from openai import OpenAI
# client = OpenAI()
import openai as client

# assistant = client.beta.assistants.create(
#   name="Investor Assistant",
#   instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#   model="gpt-4-1106-preview",
#   tools=functions
# )

assistant_id = "asst_sNRfGqBlfUQnarB6j5clyGxW"

In [3]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_9zrU6pX7oIo8xPr3vBYyEnfe', created_at=1712913888, metadata={}, object='thread')

In [5]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_Mziddn3DRXgmC1rbN0nmxLML', assistant_id='asst_sNRfGqBlfUQnarB6j5clyGxW', cancelled_at=None, completed_at=None, created_at=1712913895, expires_at=1712914495, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_9zrU6pX7oIo8xPr3vBYyEnfe', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type':

In [4]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

In [7]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy


In [6]:
get_run(run.id, thread.id).status

'requires_action'

In [12]:
get_ticker({"company_name":"Salesforce"})

"Salesforce is the world's #1 customer relationship management (CRM) platform. Learn More. Mobile Site Search. Overview. About; Stock Information. Stock Chart; ... Current Stock quote. Historical stock quote. Look Up. Investment calculator. Investment amount ($) Start date. End date. Compare to: S&P 500. Nasdaq 100. Dow 30. Other. Reinvest ... Stock analysis for Salesforce Inc (CRM:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. ALLEN, Texas-- (BUSINESS WIRE)--Social Impact Copilot™ seamlessly integrates with Salesforce Einstein Copilot to help businesses align with customers. Get a real-time Salesforce, Inc. (CRM) stock price quote with breaking news, financials, statistics, charts and more. See the latest Salesforce stock price NYSE: CRM stock ratings, related news, valuation, dividends and more to help you make your investing decisions. The latest Salesforce stock prices, stock quotes, news, and CRM history to help you in